In [ ]:
!pip install transformers
import pandas as pd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.3 MB/s eta 0:00:00


In [ ]:
data=pd.read_excel('d.xlsx')
d_typo=pd.read_excel('Typo.xlsx')

In [ ]:
d_spell=pd.read_excel('spelling.xlsx')
d_ambi=pd.read_excel('ambiguity.xlsx')

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(d_ambi, test_size=0.7, random_state=38)
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=38)

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
val_df['Sentence']=val_df['Sentence'].astype(str)
val_df = val_df.dropna(subset=['Label_id'])
data

,Sentence,Label_id,Language
0,নীতিগুলি সকল অভিবাসীদের জন্য সামাজিক ও অর্থনৈত...,7,Bengali
1,Commission income rose by 25.7 % to EUR 16.1 m...,12,English
2,The operating profit for Grain Trading increas...,12,English
3,స్వలింగ వివాహాలు అభివృద్ధి చెందుతున్న సాంస్కృత...,1,Telugu
4,स्वदेशी तथा विदेशीसँग ऋण लिएर सरकारहरू चल्न था...,5,Nepali
...,...,...,...
994,Resource managers can reassign tasks modify da...,12,English
995,স্বাস্থ্য এবং নিরাপত্তা এবং অভিবাসন সম্পর্কে ক...,7,Bengali
996,सरकारले नयाँ राहत दिन नसक्ने अवस्था उत्पन्न भय...,5,Nepali
997,Finnish Talentum reports its operating profit ...,12,English


In [ ]:
from sklearn.preprocessing import LabelEncoder

# create an instance of the LabelEncoder
label_encoder = LabelEncoder()
train_s=train_df['Label_id']
val_s=val_df['Label_id']
test_s=train_df['Label_id']
# fit the encoder to your labels and transform them to numerical labels
train_labels = label_encoder.fit_transform(train_s)
val_labels = label_encoder.transform(val_s)
test_labels = label_encoder.transform(test_s)

# check the unique values of your encoded labels
print(set(train_labels))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Load the mBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

# Define the maximum sequence length (this should match the value used during training)
max_length = 75

# Prepare the input data
def prepare_input(text, language):
    # Concatenate the text and language information
    input_text = f"{text} [{language}]"

    # Encode the concatenated string using the tokenizer
    encoded = tokenizer(
        input_text,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding='max_length'
    )

    return encoded


# Load your dataset
import pandas as pd


# Prepare the input data for prediction
texts = data['Sentence'].tolist()
languages = data['Language'].tolist()

encoded = [prepare_input(text, lang) for text, lang in zip(texts, languages)]
input_ids = tf.stack([enc['input_ids'] for enc in encoded])
attention_mask = tf.stack([enc['attention_mask'] for enc in encoded])
languages = tf.constant(languages, shape=(len(languages), 1))

# Load the pre-trained model
model_name = 'bert-base-multilingual-cased'
num_labels = 12
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Predict the class probabilities for each input
outputs = model({'input_ids': input_ids, 'attention_mask': attention_mask})
logits = outputs.logits
class_probs = tf.nn.softmax(logits, axis=-1)

# Get the predicted class for each input
predicted_classes = tf.argmax(class_probs, axis=-1)

# Convert the predicted class tensor to a numpy array
predicted_classes = predicted_classes.numpy()

# Print the predicted classes
#print(predicted_classes)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
k=data['Label_id']
# Compute the accuracy of the model predictions
num_correct = (predicted_classes == k).sum()
accuracy = num_correct / len(train_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Accuracy: 0.49760765550239233


In [ ]:
from sklearn.metrics import recall_score

# k: true labels
# predicted_classes: predicted labels
recall = recall_score(k, predicted_classes, average='macro')
print('Recall:', recall)


Recall: 0.08333333333333333


In [ ]:
from sklearn.metrics import precision_score

k = data['Label_id']
precision = precision_score(k, predicted_classes, average=None)

# If you want to compute the micro/macro/weighted average precision across all classes, you can set the 'average' parameter:
precision_micro = precision_score(k, predicted_classes, average='micro')
precision_macro = precision_score(k, predicted_classes, average='macro')
precision_weighted = precision_score(k, predicted_classes, average='weighted')
print(precision_micro)
print(precision_macro)
print(precision_weighted)

0.1041041041041041
0.008675342008675343
0.010837664491318144


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from transformers import XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification
from transformers import XLMRobertaTokenizerFast

tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')



# Define the maximum sequence length (this should match the value used during training)
max_length = 128

# Prepare the input data
def prepare_input(text):
    # Encode the text using the tokenizer
    encoded = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding='max_length'
    )

    return encoded

# Load your dataset and prepare the input data for prediction
# ...

# Load the pre-trained model
model_name = 'xlm-roberta-base'
num_labels = 12
model = TFXLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Predict the class probabilities for each input
outputs = model({'input_ids': input_ids, 'attention_mask': attention_mask})
logits = outputs.logits
class_probs = tf.nn.softmax(logits, axis=-1)

# Get the predicted class for each input
predicted_classes = tf.argmax(class_probs, axis=-1)

# Convert the predicted class tensor to a numpy array
predicted_classes = predicted_classes.numpy()

# Print the predicted classes
#print(predicted_classes)


All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

Some layers of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
k=data['Label_id']
# Compute the accuracy of the model predictions
num_correct = (predicted_classes == k).sum()
accuracy = num_correct / len(train_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Accuracy: 0.1722488038277512


In [ ]:
#@title Robustness check on the model : typo errors


In [ ]:
import pandas as pd
import random

# Function to add random typos to a word
def add_typos(word):
    # List of possible typo characters
    typo_chars = 'abcdefghijklmnopqrstuvwxyz'
    # Choose a random number of typos to add (up to 2)
    num_typos = random.randint(0, 2)
    # Add typos to the word
    for i in range(num_typos):
        # Choose a random position in the word to add the typo
        pos = random.randint(0, len(word)-1)
        # Choose a random typo character
        typo_char = random.choice(typo_chars)
        # Add the typo character to the word
        word = word[:pos] + typo_char + word[pos+1:]
    return word

# Read the Excel file into a pandas dataframe
df = pd.read_excel('d.xlsx')

# Iterate over each cell in the dataframe
for index, row in df.iterrows():
  if isinstance(df['Sentence'], str):
            # Add typos to the text in the cell
    df.at[index, df['Sentence']] = add_typos(df['Sentence'])

# Write the modified dataframe to a new Excel file
df.to_excel('Typo.xlsx', index=False)


In [ ]:
#@title Robustness check on the model : Spelling error

In [ ]:
!pip install pyahocorasick


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
import ahocorasick

# Load dictionary of correct words
with open('dict.txt') as f:
    words = set(f.read().split())

# Create Aho-Corasick automaton for fast pattern matching
AC = ahocorasick.Automaton()
for word in words:
    AC.add_word(word, word)
AC.make_automaton()

# Function to add random spelling error to a word
def add_spelling_error(word):
    # Find all possible corrections for the word
    corrections = set()
    for end_index, _ in AC.iter(word):
        corrections.add(word[:end_index])
    if len(corrections) == 0:
        # No corrections found, so return the original word
        return word
    # Choose a random correction for the word
    new_word = random.choice(list(corrections))
    # Add a random letter deletion, insertion, or substitution
    operation = random.choice(['delete', 'insert', 'substitute'])
    pos = random.randint(0, len(new_word)-1)
    if operation == 'delete' and len(new_word) > 1:
        new_word = new_word[:pos] + new_word[pos+1:]
    elif operation == 'insert':
        new_word = new_word[:pos] + random.choice('abcdefghijklmnopqrstuvwxyz') + new_word[pos:]
    elif operation == 'substitute':
        new_word = new_word[:pos] + random.choice('abcdefghijklmnopqrstuvwxyz') + new_word[pos+1:]
    return new_word

# Read the Excel file into a pandas dataframe
df = pd.read_excel('d.xlsx')

# Iterate over each cell in the dataframe
for index, row in df.iterrows():
  if isinstance(df['Sentence'], str):
            # Add typos to the text in the cell
    df.at[index, df['Sentence']] = add_typos(df['Sentence'])

# Write the modified dataframe to a new Excel file
df.to_excel('spelling.xlsx', index=False)


In [ ]:
#@title Robustness check on the model : Ambiguity

In [ ]:
import pandas as pd
import random

# Define a set of ambiguous values for each column
ambiguous_values = {
    'column1': {
        'topic': ['subject', 'summary', 'matter'],
        'marriage': ['wedding', 'cermony', 'nuptial'],
        'football': ['soccer', 'rudbey', 'grid game'],
        'immigration': ['crossing', 'migration', 'departure'],
        'advocacy': ['approval', 'defense', 'endorsement'],
        # add more values as needed
    },
   
}

# Read the Excel dataset using Pandas
df = pd.read_excel('d.xlsx')

# Loop through each cell in the dataset and add ambiguity
for index, row in df.iterrows():
    for col in df.columns:
        # Generate a random number between 0 and 1
        rand_num = random.random()
        if rand_num < 0.2:
            # Replace the cell value with a random ambiguous value
            ambiguous_values_for_col = ambiguous_values.get(col, {})
            ambiguous_values_for_cell = ambiguous_values_for_col.get(row[col], [])
            if ambiguous_values_for_cell:
                df.at[index, col] = random.choice(ambiguous_values_for_cell)

# Write the modified dataset back to Excel
df.to_excel('ambiguity.xlsx', index=False)
